In [ ]:
import requests
import re
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

In [ ]:
url_list = [f'https://www.khug.or.kr/jeonse/web/s07/s070102.jsp?cur_page={i}' for i in range(1, 71)]

# url에서 가져온 html에서 class_name이 'mb d_list'인 table을 찾아서 df로 만들고 리스트를 concat\
df_list = []
for url in tqdm(url_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    for i, tr in enumerate(table.find_all('tr')):
        if i == 0:
            columns = [th.text for th in tr.find_all('th')]+['안심전세포털']
            df_tmp = pd.DataFrame(columns=columns)
        else:
            values = [td.text for td in tr.find_all('td')]
            link = 'https://www.khug.or.kr/jeonse/web/s07/'+tr.find('a')['href']
            values.append(link)
            df_tmp = pd.concat([df_tmp, pd.DataFrame([values], columns=columns)], ignore_index=True)
    df_list.append(df_tmp)

df = pd.concat(df_list, ignore_index=True)
df.drop(columns=['청약 접수기간', '번호', '공고일자'], inplace=True)
df.rename(columns={'전용면적(m2)':'전용면적','임대보증금액':'보증금'}, inplace=True)
df = df.applymap(lambda x: x.replace('  ', '').strip() if isinstance(x, str) else x)

In [6]:
df['신청자수'] = df['신청자수'].astype(int)
df.sort_values('신청자수', ascending=False).head(20)

,시도,시군구,주소,주택유형,매입유형,전용면적,보증금,신청자수,안심전세포털
695,서울특별시,서울 송파구,서울 송파구 잠실동 219-1 아크로빌 3층 301호,다세대주택,경매등매입형,29.97,"270,900,000",137,https://www.khug.or.kr/jeonse/web/s07/s070103....
23,서울특별시,서울 강동구,"서울 강동구 길동 387-5, 387-10 길동청광플러스원큐브1차 2층 206호",오피스텔(주거용),경매등매입형,19.925,"88,200,000",75,https://www.khug.or.kr/jeonse/web/s07/s070103....
663,서울특별시,서울 성북구,"서울 성북구 하월곡동 90-1261, 90-162, 90-164, 90-165, 9...",오피스텔(주거용),경매등매입형,65.59,"227,700,000",67,https://www.khug.or.kr/jeonse/web/s07/s070103....
649,서울특별시,서울 강동구,서울 강동구 둔촌동 441-1 라온스토리 3층 301호,다세대주택,경매등매입형,28.98,"220,500,000",61,https://www.khug.or.kr/jeonse/web/s07/s070103....
206,서울특별시,서울 서초구,서울 서초구 신원동 659 해피트리앤 2층 229호,오피스텔(주거용),경매등매입형,21.168,"135,900,000",60,https://www.khug.or.kr/jeonse/web/s07/s070103....
207,서울특별시,서울 중랑구,서울 중랑구 묵동 141 천우네오젠 101동 3층 302호,다세대주택,경매등매입형,29.9,"135,900,000",43,https://www.khug.or.kr/jeonse/web/s07/s070103....
168,서울특별시,서울 도봉구,서울 도봉구 쌍문동 137-109 아리따움빌 5층 501호,다세대주택,경매등매입형,42.546,"129,600,000",41,https://www.khug.or.kr/jeonse/web/s07/s070103....
677,인천광역시,인천 연수구,인천 연수구 송도동 316 힐스테이트송도더테라스 108동 16층 1604호,오피스텔(주거용),협의매입형,84.9692,"241,200,000",41,https://www.khug.or.kr/jeonse/web/s07/s070103....
684,서울특별시,서울 중랑구,"서울 중랑구 면목동 116-2, 116-3, 116-4, 116-5, 116-6, ...",오피스텔(주거용),경매등매입형,44.64,"244,800,000",40,https://www.khug.or.kr/jeonse/web/s07/s070103....
657,서울특별시,서울 관악구,"서울 관악구 봉천동 100-184, 100-185, 100-186, 100-187 ...",다세대주택,경매등매입형,37.78,"225,900,000",36,https://www.khug.or.kr/jeonse/web/s07/s070103....


In [ ]:
# 주택 정보 전처리리
df['뒷주소'] = df['주소'].apply(lambda x: ' '.join(x.split(' ')[-4:]))
df['호'] = df['뒷주소'].str.extract(r'(\d{2,4}호)')
df['동'] = df['뒷주소'].str.extract(r'(\d{1,4}동|[가나다라마]동|[ABCDEFGHIJ]동)')
df['층'] = df['뒷주소'].str.extract(r'(\d{1,4}층)')

def get_first_non_null(row):
    if all([pd.notna(row[col]) for col in ['동', '층', '호']]):
        dong = row['뒷주소'].find(row['동'])
        floor = row['뒷주소'].find(row['층'])

        if dong < floor:
            return row['동']
        else:
            return row['층']

    for col in ['동', '층', '호']:
        if pd.notna(row[col]):
            return row[col]
    return None

df['주택명 구분자'] = df.apply(get_first_non_null, axis=1)

def extract_word_before_separator_regex(row):
    if pd.isna(row['주택명 구분자']):
        return None
    
    address = str(row['주소'])
    separator = str(row['주택명 구분자'])
    
    # 구분자 앞의 단어를 찾는 정규표현식
    pattern = r'(\S+)\s*' + re.escape(separator)
    match = re.search(pattern, address)
    
    return match.group(1) if match else None

df['주택명'] = df.apply(extract_word_before_separator_regex, axis=1)
df['주소'] = df.apply(lambda row: row['주소'][:row['주소'].find(row['주택명'])] if pd.notna(['주택명']) else row['주소'], axis=1)

df = df[['시도','시군구','주소','주택명','동','층','호','주택유형','매입유형','전용면적','보증금','안심전세포털']]

In [ ]:
df.to_excel('rent_house_list.xlsx', index=False)